# Półprzewodniki

<ol>
    <li>[Biblioteki i ustawienia](#k_rozruch)</li>
    <li>[Wczyt danych](#k_wczyt)</li>
    <li>[Mielenie](#k_mielenie)</li>
    <li>[Model](#k_model)</li>
    <li>[Eksport](#k_eksport)</li>
</ol>

<a id='k_rozruch'></a>
## Biblioteki i ustawienia

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [2]:
SEED = 26

<a id='k_wczyt'></a>
## Wczyt danych

In [3]:
sciezka = 'C:/Users/Szamil/Documents/Projekty/Dejta_Sajens/K06_polprzewodniki/'
plik = 'train.csv'

train_df = pd.read_csv(sciezka + plik)

plik = 'test.csv'

test_df = pd.read_csv(sciezka + plik)

In [4]:
train_df.head()

,id,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree,formation_energy_ev_natom,bandgap_energy_ev
0,1,33,80.0,0.6250,0.3750,0.000,9.9523,8.5513,9.1775,90.0026,90.0023,90.0017,0.0680,3.4387
1,2,194,80.0,0.6250,0.3750,0.000,6.1840,6.1838,23.6287,90.0186,89.9980,120.0025,0.2490,2.9210
2,3,227,40.0,0.8125,0.1875,0.000,9.7510,5.6595,13.9630,90.9688,91.1228,30.5185,0.1821,2.7438
3,4,167,30.0,0.7500,0.0000,0.250,5.0036,5.0034,13.5318,89.9888,90.0119,120.0017,0.2172,3.3492
4,5,194,80.0,0.0000,0.6250,0.375,6.6614,6.6612,24.5813,89.9960,90.0006,119.9893,0.0505,1.3793


In [5]:
train_df.shape, test_df.shape

((2400, 14), (600, 12))

In [6]:
train_df.info(True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 14 columns):
id                            2400 non-null int64
spacegroup                    2400 non-null int64
number_of_total_atoms         2400 non-null float64
percent_atom_al               2400 non-null float64
percent_atom_ga               2400 non-null float64
percent_atom_in               2400 non-null float64
lattice_vector_1_ang          2400 non-null float64
lattice_vector_2_ang          2400 non-null float64
lattice_vector_3_ang          2400 non-null float64
lattice_angle_alpha_degree    2400 non-null float64
lattice_angle_beta_degree     2400 non-null float64
lattice_angle_gamma_degree    2400 non-null float64
formation_energy_ev_natom     2400 non-null float64
bandgap_energy_ev             2400 non-null float64
dtypes: float64(12), int64(2)
memory usage: 262.6 KB


In [7]:
#test_df.info(True, null_counts=True)

<a id='k_mielenie'></a>
## Mielenie

### Dodanie szczegółowych

In [8]:
def wczyt(t,i):
    
    sciezka = 'C:/Users/Szamil/Documents/Projekty/Dejta_Sajens/K06_polprzewodniki/' + t + '/' + str(i) + '/'
    plik = 'geometry.xyz'

    kolumny = ['co','x','y','z','pierwiastek']
    geometry_df = pd.read_csv(sciezka + plik, sep=' ', skiprows=3, header=None, names=kolumny)

    return geometry_df

In [9]:
def wyliczAggr(i, df):
    
    lv1x = df.iloc[0,1]
    lv1y = df.iloc[0,2]
    lv1z = df.iloc[0,3]
    lv2x = df.iloc[1,1]
    lv2y = df.iloc[1,2]
    lv2z = df.iloc[1,3]
    lv3x = df.iloc[2,1]
    lv3y = df.iloc[2,2]
    lv3z = df.iloc[2,3]

    pierwiastek_avg_x = df.groupby(['pierwiastek'])['x'].mean()
    pierwiastek_avg_y = df.groupby(['pierwiastek'])['y'].mean()
    pierwiastek_avg_z = df.groupby(['pierwiastek'])['z'].mean()
    pierwiastek_std_x = df.groupby(['pierwiastek'])['x'].std()
    pierwiastek_std_y = df.groupby(['pierwiastek'])['y'].std()
    pierwiastek_std_z = df.groupby(['pierwiastek'])['z'].std()

    co_avg_x = df.groupby(['co'])['x'].mean()
    co_avg_y = df.groupby(['co'])['y'].mean()
    co_avg_z = df.groupby(['co'])['z'].mean()
    co_std_x = df.groupby(['co'])['x'].std()
    co_std_y = df.groupby(['co'])['y'].std()
    co_std_z = df.groupby(['co'])['z'].std()

    lv_x_avg = co_avg_x.loc['lattice_vector']
    atom_x_avg = co_avg_x.loc['atom']
    lv_y_avg = co_avg_y.loc['lattice_vector']
    atom_y_avg = co_avg_y.loc['atom']
    lv_z_avg = co_avg_z.loc['lattice_vector']
    atom_z_avg = co_avg_z.loc['atom']

    lv_x_std = co_std_x.loc['lattice_vector']
    atom_x_std = co_std_x.loc['atom']
    lv_y_std = co_std_y.loc['lattice_vector']
    atom_y_std = co_std_y.loc['atom']
    lv_z_std = co_std_z.loc['lattice_vector']
    atom_z_std = co_std_z.loc['atom']

    try:
        al_x_avg = pierwiastek_avg_x.loc['Al']
        al_x_std = pierwiastek_std_x.loc['Al']
        al_y_avg = pierwiastek_avg_y.loc['Al']
        al_y_std = pierwiastek_std_y.loc['Al']
        al_z_avg = pierwiastek_avg_z.loc['Al']
        al_z_std = pierwiastek_std_z.loc['Al']
    except:
        al_x_avg = np.nan
        al_x_std = np.nan
        al_y_avg = np.nan
        al_y_std = np.nan
        al_z_avg = np.nan
        al_z_std = np.nan
    try:
        ga_x_avg = pierwiastek_avg_x.loc['Ga']
        ga_x_std = pierwiastek_std_x.loc['Ga']
        ga_y_avg = pierwiastek_avg_y.loc['Ga']
        ga_y_std = pierwiastek_std_y.loc['Ga']
        ga_z_avg = pierwiastek_avg_z.loc['Ga']
        ga_z_std = pierwiastek_std_z.loc['Ga']
    except:
        ga_x_avg = np.nan
        ga_x_std = np.nan
        ga_y_avg = np.nan
        ga_y_std = np.nan
        ga_z_avg = np.nan
        ga_z_std = np.nan
    try:
        o_x_avg = pierwiastek_avg_x.loc['O']
        o_x_std = pierwiastek_std_x.loc['O']
        o_y_avg = pierwiastek_avg_y.loc['O']
        o_y_std = pierwiastek_std_y.loc['O']
        o_z_avg = pierwiastek_avg_z.loc['O']
        o_z_std = pierwiastek_std_z.loc['O']
    except:    
        o_x_avg = np.nan
        o_x_std = np.nan
        o_y_avg = np.nan
        o_y_std = np.nan
        o_z_avg = np.nan
        o_z_std = np.nan

    return (i, lv1x, lv1y, lv1z, lv2x, lv2y, lv2z, lv3x, lv3y, lv3z, lv_x_avg, atom_x_avg, lv_y_avg, atom_y_avg, lv_z_avg, atom_z_avg, lv_x_std, atom_x_std, lv_y_std, atom_y_std, lv_z_std, atom_z_std, al_x_avg, ga_x_avg, o_x_avg, al_y_avg, ga_y_avg, o_y_avg, al_z_avg, ga_z_avg, o_z_avg, al_x_std, ga_x_std, o_x_std, al_y_std, ga_y_std, o_y_std, al_z_std, ga_z_std, o_z_std)

In [10]:
train_geo_aggr_list = []

for p in range(0, len(train_df)):
    i = p+1
    tmp_geometry_df = wczyt('train', i)
    tmp_aggr = wyliczAggr(i, tmp_geometry_df)
    train_geo_aggr_list.append(tmp_aggr)

test_geo_aggr_list = []
    
for p in range(0, len(test_df)):
    i = p+1
    tmp_geometry_df = wczyt('test', i)
    tmp_aggr = wyliczAggr(i, tmp_geometry_df)
    test_geo_aggr_list.append(tmp_aggr)

    
kolumny = ['id', 'lv1x', 'lv1y', 'lv1z', 'lv2x', 'lv2y', 'lv2z', 'lv3x', 'lv3y', 'lv3z', 'lv_x_avg', 'atom_x_avg', 'lv_y_avg', 'atom_y_avg', 'lv_z_avg', 'atom_z_avg', 'lv_x_std', 'atom_x_std', 'lv_y_std', 'atom_y_std', 'lv_z_std', 'atom_z_std', 'al_x_avg', 'ga_x_avg', 'o_x_avg', 'al_y_avg', 'ga_y_avg', 'o_y_avg', 'al_z_avg', 'ga_z_avg', 'o_z_avg', 'al_x_std', 'ga_x_std', 'o_x_std', 'al_y_std', 'ga_y_std', 'o_y_std', 'al_z_std', 'ga_z_std', 'o_z_std']

train_geo_aggr_df = pd.DataFrame(train_geo_aggr_list, columns = kolumny)
test_geo_aggr_df = pd.DataFrame(test_geo_aggr_list, columns = kolumny)

In [11]:
big_train_df = train_df.merge(train_geo_aggr_df, on='id')
big_test_df = test_df.merge(test_geo_aggr_df, on='id')
big_train_df.shape, big_test_df.shape

((2400, 53), (600, 51))

Wypełnianie braków

In [12]:
big_train_df.fillna(-99, inplace=True)
big_test_df.fillna(-99, inplace=True)

In [13]:
# Podzial na uczacy, walidacyjny i testowy

X_train, X_valid, y_train, y_valid = train_test_split(
    big_train_df.loc[:, ~big_train_df.columns.isin(['id','formation_energy_ev_natom','bandgap_energy_ev']) ], 
    big_train_df.loc[:,['formation_energy_ev_natom','bandgap_energy_ev']],
    test_size=0.2, random_state=SEED)

X_test = big_test_df.iloc[:,1:]

X_train.shape, X_valid.shape, X_test.shape, y_train.shape, y_valid.shape

((1920, 50), (480, 50), (600, 50), (1920, 2), (480, 2))

### Max

In [69]:
train_minmax_list = []
    
for p in range(0, len(train_df)):
    i = p+1
    tmp_geometry_df = wczyt('train', i)
    tmp_geometry_df2 = tmp_geometry_df.loc[tmp_geometry_df['co']=='atom']
    tmp_minmax = (i, tmp_geometry_df2['x'].max(), tmp_geometry_df2['y'].max(), tmp_geometry_df2['z'].max()
                 , tmp_geometry_df2['x'].min(), tmp_geometry_df2['y'].min(), tmp_geometry_df2['z'].min())
    train_minmax_list.append(tmp_minmax)    
    
train_minmax_df = pd.DataFrame(train_minmax_list, columns = ['id','max_x','max_y','max_z','min_x','min_y','min_z'])

In [72]:
a = 2
a += 1
a

3

In [71]:
train_minmax_df.min(), train_minmax_df.max()

(id       1.000000
 max_x    0.916272
 max_y    1.374448
 max_z    4.929735
 min_x   -2.067313
 min_y   -0.277155
 min_z   -0.702511
 dtype: float64, id       2400.000000
 max_x      24.506870
 max_y       9.938675
 max_z      24.295771
 min_x       1.010880
 min_y       0.621624
 min_z       1.348378
 dtype: float64)

### 3D

In [73]:
tmp_geometry_df = wczyt('train', 4)

In [36]:
tmp_geometry_df.head()

,co,x,y,z,pierwiastek
0,lattice_vector,5.003557,0.024477,0.003776,NaN
1,lattice_vector,-2.522969,4.320664,-0.006625,NaN
2,lattice_vector,-0.013103,0.016168,13.531761,NaN
3,atom,-0.001900,0.002344,1.962105,Al
4,atom,-0.008452,0.010428,8.727986,In


In [74]:
tmp_geometry_df2 = tmp_geometry_df.loc[tmp_geometry_df['co']=='atom']

In [26]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected = True)

In [75]:
x_al, y_al, z_al = tmp_geometry_df2.loc[tmp_geometry_df2['pierwiastek']=='Al','x'], tmp_geometry_df2.loc[tmp_geometry_df2['pierwiastek']=='Al','y'], tmp_geometry_df2.loc[tmp_geometry_df2['pierwiastek']=='Al','z']

x_ga, y_ga, z_ga = tmp_geometry_df2.loc[tmp_geometry_df2['pierwiastek']=='Ga','x'], tmp_geometry_df2.loc[tmp_geometry_df2['pierwiastek']=='Ga','y'], tmp_geometry_df2.loc[tmp_geometry_df2['pierwiastek']=='Ga','z']

x_in, y_in, z_in = tmp_geometry_df2.loc[tmp_geometry_df2['pierwiastek']=='In','x'], tmp_geometry_df2.loc[tmp_geometry_df2['pierwiastek']=='In','y'], tmp_geometry_df2.loc[tmp_geometry_df2['pierwiastek']=='In','z']

x_o, y_o, z_o = tmp_geometry_df2.loc[tmp_geometry_df2['pierwiastek']=='O','x'], tmp_geometry_df2.loc[tmp_geometry_df2['pierwiastek']=='O','y'], tmp_geometry_df2.loc[tmp_geometry_df2['pierwiastek']=='O','z']

trace1 = go.Scatter3d(
    x=x_al,
    y=y_al,
    z=z_al,
    mode='markers',
    name='Al',
    marker=dict(
        size=12,
        color='blue',                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)


trace2 = go.Scatter3d(
    x=x_ga,
    y=y_ga,
    z=z_ga,
    mode='markers',
    name='Ga',
    marker=dict(
        size=12,
        color='red',                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

trace3 = go.Scatter3d(
    x=x_in,
    y=y_in,
    z=z_in,
    mode='markers',
    name='In',
    marker=dict(
        size=12,
        color='green',                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

trace4 = go.Scatter3d(
    x=x_o,
    y=y_o,
    z=z_o,
    mode='markers',
    name='O',
    marker=dict(
        size=12,
        color='yellow',                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

data = [trace1,trace2,trace3,trace4]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='3d-scatter-colorscale')

### Część Mara

In [ ]:
maxV = 28

minV = -2.3

res = 100maxV = 28

minV = -2.3

res = 100

5:56

def matrix_pos(x):

   pos_x = (x - minV)/(maxV-minV)*res

   return int(pos_x)

5:57

el_dict = {'Al':0, 'Ga':1, 'In':2, 'O':3}

A = np.zeros(3,res,res,res,4))

for p in range(0, len(train_df)):

   i = p+1

   tmp_geometry_df = wczyt('train', i)

   tmp_geometry_df2 = tmp_geometry_df.loc[tmp_geometry_df['co']=='atom']

   tmp_geometry_df2['channel'] = tmp_geometry_df2['pierwiastek'].map(el_dict)

   for _, pos in tmp_geometry_df2.iterrows():

       A[p, matrix_pos(pos.x), matrix_pos(pos.y), matrix_pos(pos.z), pos['channel']]=+1

A.shape

5:57

import matplotlib.pyplot as plt

%matplotlib inline



plt.imshow(A.sum(axis=0))

<a id='k_model'></a>
## Model

In [14]:
def rmsle(yhat,y):
    yhat = np.array(yhat)
    y = np.array(y)
    return np.sqrt(sum((np.log(yhat+1) - np.log(y+1))**2) / len(yhat))

In [15]:
def obliczBlad(y_1,y_2):
    blad_1 = rmsle(y_1,y_valid.iloc[:,0])
    blad_2 = rmsle(y_2,y_valid.iloc[:,1])
    wynik = (blad_1 + blad_2) / 2
    print ('Błąd 1:', '%.4f' % blad_1
          , 'Błąd 2:', '%.4f' % blad_2
          , 'Średnia:', '%.4f' % wynik)

Szympans

In [16]:
obliczBlad(np.ones(len(y_valid))*y_train.iloc[:,0].mean(),np.ones(len(y_valid))*y_train.iloc[:,1].mean())

Błąd 1: 0.0882 Błąd 2: 0.3365 Średnia: 0.2124


### Dżewko regre

In [17]:
from sklearn.tree import DecisionTreeRegressor

In [18]:
regr_1 = DecisionTreeRegressor(max_depth=8)
regr_1.fit(X_train, y_train.iloc[:,0])
y_1 = regr_1.predict(X_valid)

regr_2 = DecisionTreeRegressor(max_depth=8)
regr_2.fit(X_train, y_train.iloc[:,1])
y_2 = regr_2.predict(X_valid)

In [19]:
obliczBlad(y_1,y_2)

Błąd 1: 0.0415 Błąd 2: 0.1244 Średnia: 0.0830


<a id='k_eksport'></a>
## Eksport

In [20]:
#ostateczny_skoring = wybrany_model.predict(X_test_do_skoringu)

In [21]:
#do_wysylki = pd.DataFrame({
#    'PassengerId':X_test['PassengerId'],
#    'Survived':ostateczny_skoring
#})
#
#plik = 'do_wysylki.csv'
#
#do_wysylki.to_csv(sciezka + plik, index = False)